# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations

pd.set_option("display.max_rows", 100)

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [2]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
sailors_df2 = pd.read_csv("day2data.csv", header=1, index_col=0)
sailors_df3 = pd.read_csv("day3data.csv", header=1, index_col=0)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df[resource] = 0
    sailors_df2[resource] = 0
    sailors_df3[resource] = 0
sailors_df["Used"] = ""
sailors_df2["Used"] = ""
sailors_df3["Used"] = ""
columns = ["resource", "count", "amt_used", "health_bonus"]
data = np.array([["Bed", 30, 0, 5],
          ["Ventilator", 10, 0, 30],
          ["Supplemental oxygen", 10, 0, 20],
          ["Plasma", 10, 0, 15],
          ["Remdesivir", 7, 0, 30],
          ["Dexamethasone", 20, 0, 25],
          ["Casirivimab", 10, 0, 15],
          ["Chloroquine", 17, 0, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
resources_df["reusable"] = pd.Series([True, True, True, False, False, False, False, False])
print(resources_df,"\n")

              resource  count  amt_used  health_bonus  reusable
0                  Bed     30         0             5      True
1           Ventilator     10         0            30      True
2  Supplemental oxygen     10         0            20      True
3               Plasma     10         0            15     False
4           Remdesivir      7         0            30     False
5        Dexamethasone     20         0            25     False
6          Casirivimab     10         0            15     False
7          Chloroquine     17         0            10     False 



In [3]:
# Get combined bonus from adding up individual bonuses in tups
def get_bonus(tups):
    total = 0
    for name, bonus, count in tups:
        total += bonus
    return total

# Get names from list of resource tuples
def get_names(tups):
    names = []
    for name, bonus, count in tups:
        names.append(name)
    return names

# Get lowest count value from list of resource tuples
def get_min_count(tups):
    counts = []
    for name, bonus, count in tups:
        counts.append(count)
    return min(counts)

# Check if list of resource tuples contains a reusable resource
def has_reusable(tups, resources_df):
    reusable = list(resources_df.loc[resources_df["reusable"]==True]["resource"])
    for name, bonus, count in tups:
        if name in reusable:
            return True
    return False

# Reset reusable resources in DataFrame
def reset_reusable(resources_df):
    resources_df.loc[resources_df["reusable"]==True, "count"] += resources_df.loc[resources_df["reusable"]==True, "amt_used"]
    resources_df.loc[resources_df["reusable"]==True, "amt_used"] = 0

def calc_final_score(allocation_df, resources_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    resources_df = resources_df[resources_df["reusable"]==False]
    resource_bonus = (resources_df["count"]*resources_df["health_bonus"]).sum()/2
    return final_health.sum() + resource_bonus

# Verify that resource counts add up
# Does not verify that treated sailors have beds
def check_dfs(sailors_dfs, resources_df):
    for resource in resources_df.itertuples():
        used = 0
        for df in sailors_dfs:
            if resource.reusable and df[resource.resource].sum() > (resource.count+resource.amt_used):
                print(resource.resource)
                return False
            used += df[resource.resource].sum()
        if not resource.reusable and used != resource.amt_used:
            return False
    return True

In [4]:
def use_resource(sailors_df, resources_df, resource, indices):
    """
    resource = resource name as string
    indices = Pandas index list
    Ex. for single indices do [df.index[position]]
    Ex. for range of values do df.index[start:end]
    """
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] += 1
    sailors_df.loc[indices, "Total"] += bonus_dict[resource]
    sailors_df.loc[indices, "Used"] = sailors_df.loc[indices, "Used"].apply(lambda x: ','.join([resource]+x.split(',')))
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] += len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)


def ethical_boosting(sailors_df, resources_df, use_max):
    if not use_max:
        return
    nonreusable = resources_df.loc[(resources_df["reusable"]==False) & (resources_df["count"] > 0)].copy()
    
    sailor_idx = 0
    while nonreusable["count"].sum() > 0:
        sailors_df.sort_values(by=["Total"], inplace=True, ascending=True)
        unavailable = sailors_df.loc[sailors_df.index[sailor_idx], "Used"].split(',')
        nonreusable.sort_values(by=["health_bonus"], ascending=False, inplace=True)
        resource_idx = 0
        while resource_idx < len(nonreusable) and nonreusable.loc[nonreusable.index[resource_idx], "resource"] in unavailable:
            resource_idx += 1
        if resource_idx >= len(nonreusable) or sailors_df.loc[sailors_df.index[sailor_idx], "Bed"] == 0:
            sailor_idx += 1
        else:
            r = nonreusable.loc[nonreusable.index[resource_idx], "resource"]
            use_resource(sailors_df, resources_df, r, [sailors_df.index[sailor_idx]])
            sailors_df.loc[sailors_df.index[sailor_idx], "Used"] = ','.join([r]+unavailable)
            nonreusable = resources_df.loc[(resources_df["reusable"]==False) & (resources_df["count"] > 0)].copy()

            
def allocate_reusable(sailors_df, resources_df):
    count_col = resources_df.columns.get_loc("count")
    
    # Allocate beds
    bedless_sailors = sailors_df.loc[sailors_df["Bed"] == 0].sort_values(by="Total", ascending=True)
    for sailor in bedless_sailors.itertuples():
        if resources_df.loc[resources_df["resource"] == "Bed"].iat[0, count_col] == 0:
            break
        use_resource(sailors_df, resources_df, "Bed", [sailor.Index])
        
    # Allocate ventilators to sailors who have beds and no supplemental oxygen
    need_ventilator = sailors_df.loc[(sailors_df["Bed"] == 1) & (sailors_df["Supplemental oxygen"] == sailors_df["Ventilator"])]\
                                .sort_values(by="Total", ascending=True)
    for sailor in need_ventilator.itertuples():
        if resources_df.loc[resources_df["resource"] == "Ventilator"].iat[0, count_col] == 0:
            break
        use_resource(sailors_df, resources_df, "Ventilator", [sailor.Index])

    # Allocate supplemental oxygen to sailors who have beds and no supplemental oxygen
    need_oxymask = sailors_df.loc[(sailors_df["Bed"] == 1) & (sailors_df["Supplemental oxygen"] == sailors_df["Ventilator"])]\
                                .sort_values(by="Total", ascending=True)
    for sailor in need_oxymask.itertuples():
        if resources_df.loc[resources_df["resource"] == "Supplemental oxygen"].iat[0, count_col] == 0:
            break
        use_resource(sailors_df, resources_df, "Supplemental oxygen", [sailor.Index])


def allocate_dying_helper(sailors_df, resources_df, ascending_bool=True):
    dying = sailors_df.loc[sailors_df["Total"] <= 0].sort_values(by=["Total"], ascending=True)
    count_col = resources_df.columns.get_loc("count")
    bonus_col = resources_df.columns.get_loc("health_bonus")
    bed_count = resources_df.loc[resources_df["resource"]=="Bed"].iat[0, count_col]
    bed_health = resources_df.loc[resources_df["resource"]=="Bed"].iat[0, bonus_col]
    if len(dying) > bed_count:
        dying = dying[bed_count:] # Abandon sailors with the lowest totals to use all beds and conserve most resources
    dying.sort_values(by=["Total"], inplace=True, ascending=ascending_bool)
    
    for sailor in dying.itertuples():
        if resources_df.loc[resources_df["resource"]=="Bed"].iat[0, count_col] == 0:
            # Ran out of beds so no need to go through any more sailors
            # Unnecessary, but I'll keep it in case
            break
        # Make combiations with the reusable resources, but need to separate them since they can't be used together
        ventilator = resources_df[(~resources_df["resource"].isin(["Bed", "Supplemental oxygen"])) \
                                  & (resources_df["count"] > 0)]
        oxymask = resources_df[(~resources_df["resource"].isin(["Bed", "Ventilator"])) \
                               & (resources_df["count"] > 0)]
        venti_tups = list(zip(ventilator["resource"], ventilator["health_bonus"], ventilator["count"]))
        oxy_tups = list(zip(oxymask["resource"], oxymask["health_bonus"], oxymask["count"]))
        total = sailor.Total+bed_health
        mindict = {}
        # Generate combinations of resources to heal sailors. Prioritize reusable if available.
        # If two combinations will yield same result, prioritize combination with higher count resources
        for i in range(1,len(venti_tups)+1):
            for comb in combinations(venti_tups, i):
                if has_reusable(venti_tups+oxy_tups, resources_df)==has_reusable(comb, resources_df):
                    x = total + get_bonus(comb)
                    if x in mindict and get_min_count(mindict[x]) > get_min_count(comb):
                        continue
                    else:
                        mindict[x] = comb
        for i in range(1, len(oxy_tups)+1):
            for comb in combinations(oxy_tups, i):
                if has_reusable(venti_tups+oxy_tups, resources_df)==has_reusable(comb, resources_df):
                    x = total + get_bonus(comb)
                    if x in mindict and get_min_count(mindict[x]) > get_min_count(comb):
                        continue
                    else:
                        mindict[x] = comb
        # Only keep combinations that will save sailors
        minlist = [k for k in mindict.keys() if k > 0]
        if len(minlist) == 0:
            # No combination of resources can save sailor, so skip
            continue
        m = min(minlist)
        use_resource(sailors_df, resources_df, "Bed", [sailor.Index])
        for resource in get_names(mindict[m]):
                use_resource(sailors_df, resources_df, resource, [sailor.Index])


def allocate_dying(sailors_df, resources_df):
    # Make copies of dataframes
    s_df1 = sailors_df.copy()
    r_df1 = resources_df.copy()
    s_df2 = sailors_df.copy()
    r_df2 = resources_df.copy()
    
    # Run allocations using ascending and descending sorting of Totals
    allocate_dying_helper(s_df1, r_df1, True)
    allocate_dying_helper(s_df2, r_df2, False)
    
    # Calculate the score for each sorting method
    score1 = calc_final_score(s_df1, r_df1)
    score2 = calc_final_score(s_df2, r_df2)

    # Determine which method yields the higher score
    if score1 > score2:
        s_winner = s_df1
        r_winner = r_df1
    else:
        s_winner = s_df2
        r_winner = r_df2

    # Modify original dataframes with winning values
    for column in sailors_df.columns:
        sailors_df[column] = s_winner[column]
    for column in resources_df.columns:
        resources_df[column] = r_winner[column]

def allocate(sailors_df, resources_df, use_max):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df["Orig_Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df.sort_values(by=["Total"], inplace=True, ascending=False)
    
    allocate_dying(sailors_df, resources_df)
    allocate_reusable(sailors_df, resources_df)
    ethical_boosting(sailors_df, resources_df, use_max)
    
    return sailors_df, resources_df

In [5]:
a_df, r_df = allocate(sailors_df, resources_df, False)
print(check_dfs([a_df], r_df))

reset_reusable(r_df)
a_df2, r_df2 = allocate(sailors_df2, r_df, False)
print(check_dfs([a_df, a_df2], r_df2))

reset_reusable(r_df2)
a_df3, r_df3 = allocate(sailors_df3, r_df2, True)
print(check_dfs([a_df, a_df2, a_df3], r_df3))

print(calc_final_score(a_df, r_df)+calc_final_score(a_df2, r_df2)+calc_final_score(a_df3, r_df3))
print(r_df3)
a_df3


True
True
True
4716.5
              resource  count  amt_used  health_bonus  reusable
0                  Bed      0        30             5      True
1           Ventilator      0        10            30      True
2  Supplemental oxygen      0        10            20      True
3               Plasma      0        10            15     False
4           Remdesivir      0         7            30     False
5        Dexamethasone      0        20            25     False
6          Casirivimab      0        10            15     False
7          Chloroquine      0        17            10     False


,Health,Rate of health decline,Unnamed: 3,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Used,Total,Orig_Total
ID,,,,,,,,,,,,,,
81,55,-98,NaN,1,1,1,0,0,0,0,1,"Remdesivir,Chloroquine,Ventilator,Bed,",32,-43
82,57,-99,NaN,1,1,1,0,0,0,0,1,"Remdesivir,Chloroquine,Ventilator,Bed,",33,-42
85,57,-99,NaN,1,1,1,0,0,0,0,1,"Remdesivir,Chloroquine,Ventilator,Bed,",33,-42
77,24,-90,NaN,1,1,0,1,1,1,0,1,"Casirivimab,Plasma,Chloroquine,Dexamethasone,V...",34,-66
63,37,-50,NaN,1,0,0,1,0,0,1,1,"Chloroquine,Dexamethasone,Supplemental oxygen,...",47,-13
65,57,-55,NaN,1,1,0,0,0,0,0,0,"Ventilator,Bed,",37,2
75,89,-86,NaN,1,1,0,0,0,0,0,0,"Ventilator,Bed,",38,3
66,48,-60,NaN,1,0,0,1,0,0,1,0,"Dexamethasone,Supplemental oxygen,Bed,",38,-12
49,34,-25,NaN,1,0,0,1,0,0,0,0,"Dexamethasone,Bed,",39,9


In [6]:
a_df.drop(["Used"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day1Submission.csv")
a_df2.drop(["Unnamed: 3", "Used"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day2Submission.csv")
a_df3.drop(["Unnamed: 3", "Used"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day3Submission.csv")